# Генератор текста

## **Задача:**

Обучить нейронную сеть для создания названия текста.

* В качеств текстов взяты стихотворения Корнея Чуковского
* Обученная моедль будет генерировать и добавлять слова к предложенной фразе

## Подготовка данных <a name="Подготовка-данных"></a>

In [2]:
import sys; print(sys.version)

import pandas as pd
import numpy as np
from IPython.display import display

from pymystem3 import Mystem

import re 
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords

import torch
import transformers
import transformers as ppb
# pytorch transformers

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import warnings
warnings.simplefilter("ignore")

3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 02:25:08) 
[GCC 7.5.0]


In [120]:
#загружаем данные, делаем первый взгляд на таблицу
data = pd.read_csv('poems_title.csv') 
display(data.head(10))

,text,poem_title
0,"[<p>1</p>, <p>Добрый доктор Айболит!<br/>\nОн ...",Айболит
1,"[<p>Одеяло<br/>\nУбежало,<br/>\nУлетела просты...",Мойдодыр
2,[<p>Солнце по небу гуляло<br/>\nИ за тучу забе...,Краденое солнце
3,[<p>Как у нашего Мирона<br/>\nНа носу сидит во...,Чудо-дерево
4,"[<p>I</p>, <p>Маленькие дети!<br/>\nНи за что ...",Бармалей
5,"[<p>1</p>, <p>Скачет сито по полям,<br/>\nА ко...",Федорино горе
6,"[<p><strong>Часть первая</strong></p>, <p>Ехал...",Тараканище
7,[<p>Замяукали котята:<br/>\n«Надоело нам мяука...,Путаница
8,"[<p><strong>Часть первая</strong></p>, <p>1</p...",Крокодил
9,"[<p>Муха, Муха — Цокотуха,<br/>\nПозолоченное ...",Муха-Цокотуха


In [121]:
#Подготавливаем тексты к лемматизации, удаляем ненужные символы
def clear_text(text):
    new_text = text.replace("</p>,", '')
    new_text = re.sub(r'[^а-яА-ЯёЁ.,!?—:;]', ' ', new_text)
    return ' '.join(new_text.split())

data['text'] = data['text'].apply(clear_text)
print(data.head())

                                                text       poem_title
0  Добрый доктор Айболит! Он под деревом сидит. П...          Айболит
1  Одеяло Убежало, Улетела простыня, И подушка, К...         Мойдодыр
2  Солнце по небу гуляло И за тучу забежало. Глян...  Краденое солнце
3  Как у нашего Мирона На носу сидит ворона. А на...      Чудо-дерево
4  Маленькие дети! Ни за что на свете Не ходите в...         Бармалей


In [122]:
#восмотрим на самые длинные стихотворения по количеству слов
def len_text(text):
    return len(text)
data['len_text'] = data['text'].apply(len_text)
data['len_text'].sort_values()

37      109
17      143
26      151
16      161
27      164
29      168
20      183
36      194
34      206
32      213
41      231
18      277
35      287
19      356
31      394
14      395
15      398
38      456
30      597
24      910
13     1018
39     1769
33     1873
25     2345
23     2429
22     2584
11     2777
28     2879
12     3091
21     3140
3      3607
7      4114
9      4787
10     5274
2      5709
1      6167
5      6261
6      6308
4      6398
0      7007
8     14717
40    18766
Name: len_text, dtype: int64

## <b>Проведем обучение моделей</b>

* Обучать модель будем не на всем наборе данных, так как не позволяют машинные мощности. Возьмем одну строку == одно стихотворение. Стихотворение под названием "Ленинградским детям"

In [118]:
min_len = data['len_text'].min()
min_text = data.loc[data.index == 13]
display(min_text)

,text,poem_title,len_text
13,Стих для взрослых Промчатся над вами Года за г...,Ленинградским детям,1018


In [124]:
tokenizer = Tokenizer() #instantiating the tokenizer
sentence = data['text'][13]
corpus = sentence.lower().split("\n") #converting the sentence to lowercase
tokenizer.fit_on_texts(corpus) #creates tokens for each words 
total_words = len(tokenizer.word_index) + 1 #calculating total number of words in the initial sentence
print(total_words)

126


Количество слов в выбранном стихотворении равно 126.

In [125]:
input_sequences = [] #training features (x) will be a list

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0] #converts each sentence as its tokenized equivalent
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1] #generating n gram sequences
        input_sequences.append(n_gram_sequence) #appending each n gram sequence to the list of our features (xs)
print(input_sequences)

[[23, 24], [23, 24, 25], [23, 24, 25, 26], [23, 24, 25, 26, 27], [23, 24, 25, 26, 27, 9], [23, 24, 25, 26, 27, 9, 28], [23, 24, 25, 26, 27, 9, 28, 29], [23, 24, 25, 26, 27, 9, 28, 29, 30], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31, 32], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31, 32, 3], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31, 32, 3, 33], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31, 32, 3, 33, 12], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31, 32, 3, 33, 12, 34], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31, 32, 3, 33, 12, 34, 35], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31, 32, 3, 33, 12, 34, 35, 3], [23, 24, 25, 26, 27, 9, 28, 29, 30, 1, 10, 3, 11, 31, 32, 3, 33, 12, 34, 35, 3, 1], 

In [126]:
max_sequence_len = max([len(x) for x in input_sequences]) #calculating the length of the longest sequence
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')) #pre-pading each value of the input_sequence
xs, labels = input_sequences[:,:-1],input_sequences[:,-1] #creating xs and their labels using numpy slicing
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words) #creating one hot encoding values
print(xs)
print(ys)

[[  0   0   0 ...   0   0  23]
 [  0   0   0 ...   0  23  24]
 [  0   0   0 ...  23  24  25]
 ...
 [  0   0  23 ... 122   1 123]
 [  0  23  24 ...   1 123 124]
 [ 23  24  25 ... 123 124   9]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [127]:
print(labels)

[ 24  25  26  27   9  28  29  30   1  10   3  11  31  32   3  33  12  34
  35   3   1  36   1  13  37  38  39   6   4  14  40   1  41  42  43  44
   1   7  45  46  47  48   1  13  49  50   7   6   4  51  52   1  53  15
  54  15   5  55  14  56  16  57   1  58  59   7  17  60  16  61  62  63
   6  10   3  11  64  65  66  67   1  68  69  70  71  72  73  18   4   2
  74  75  76  77  78  79   1  80   5  81   8  17  82   2  83  84  85  86
  87   5  88  89   2  19  20   8  90   2  19  20  12   2  91   4  92  93
   2  94  95   8   2  96  21  18  97  98  99   3  21 100 101 102 103 104
 105 106   1 107   5 108 109 110   1 111 112 113 114 115   2 116 117 118
 119   2 120  22   3 121   2  22 122   1 123 124   9 125]


**Обучение модели нейронной сети**

In [128]:
model = Sequential() #creating a sequential model
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1)) #adding an embedding layer with 64 as the embedding dimension
model.add(Bidirectional(LSTM(20))) #adding 20 LSTM units

#Слои, которые пробовались, но не дали лучшего результата
#model.add(Dropout(0.2))
#model.add(Flatten())
#model.add(Dense(total_words, activation='relu')) #creating a dense layer with 54 output units (total_words) with softmax activation

model.add(Dense(total_words, activation='softmax'))

In [129]:
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #compiling the model with adam optimiser
history = model.fit(xs, ys, epochs=200, verbose=1) #training for 500 epochs

Train on 176 samples
Epoch 1/200
176/176 [==============================] - 7s 39ms/sample - loss: 4.8374 - accuracy: 0.0057
Epoch 2/200
176/176 [==============================] - 3s 16ms/sample - loss: 4.8253 - accuracy: 0.0739
Epoch 3/200
176/176 [==============================] - 3s 15ms/sample - loss: 4.8158 - accuracy: 0.0852
Epoch 4/200
176/176 [==============================] - 3s 15ms/sample - loss: 4.8025 - accuracy: 0.1023
Epoch 5/200
176/176 [==============================] - 3s 15ms/sample - loss: 4.7854 - accuracy: 0.1080
Epoch 6/200
176/176 [==============================] - 3s 15ms/sample - loss: 4.7584 - accuracy: 0.0852
Epoch 7/200
176/176 [==============================] - 3s 15ms/sample - loss: 4.7194 - accuracy: 0.0682
Epoch 8/200
176/176 [==============================] - 3s 14ms/sample - loss: 4.6590 - accuracy: 0.0568
Epoch 9/200
176/176 [==============================] - 3s 15ms/sample - loss: 4.6129 - accuracy: 0.0568
Epoch 10/200
176/176 [=====================

176/176 [==============================] - 2s 14ms/sample - loss: 1.6866 - accuracy: 0.6250
Epoch 156/200
176/176 [==============================] - 3s 14ms/sample - loss: 1.6806 - accuracy: 0.6250
Epoch 157/200
176/176 [==============================] - 3s 15ms/sample - loss: 1.6718 - accuracy: 0.6307
Epoch 158/200
176/176 [==============================] - 2s 14ms/sample - loss: 1.6627 - accuracy: 0.6477
Epoch 159/200
176/176 [==============================] - 2s 14ms/sample - loss: 1.6541 - accuracy: 0.6364
Epoch 160/200
176/176 [==============================] - 2s 14ms/sample - loss: 1.6471 - accuracy: 0.6307
Epoch 161/200
176/176 [==============================] - 3s 14ms/sample - loss: 1.6434 - accuracy: 0.6420
Epoch 162/200
176/176 [==============================] - 3s 15ms/sample - loss: 1.6346 - accuracy: 0.6307
Epoch 163/200
176/176 [==============================] - 3s 14ms/sample - loss: 1.6276 - accuracy: 0.6534
Epoch 164/200
176/176 [==============================] - 3s 

In [134]:
#predicting the next word using an initial sentence
input_phrase = 'Ленинград детям'
output = ""
next_words = 2
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([input_phrase])[0] #converting our input_phrase to tokens and excluding the out of vcabulary words
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') #padding the input_phrase
    predicted = model.predict_classes(token_list, verbose=0) #predicting the token of the next word using our trained model
    output_word = " " #initialising output word as blank at the beginning
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word #converting the token back to the corresponding word and storing it in the output_word
            break
    input_phrase += " " + output_word
    output += " " + output_word
print(output)
print(input_phrase)

 для взрослых
Ленинград детям для взрослых


## Выводы

* В итоге получилось такое странное название: **Ленинград детям для взрослых** вместо "Ленинградским детям".
* Если бы модель обучилась на большем количестве слов, то результат был бы более умным.

Однако модель работает и имея большие технические мощности, можно обучить ее получше=)